In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def read_data(filename):
    data = pd.read_csv(filename, na_values=['-nan(ind)', ' -nan(ind)'])
    data = data.astype({
        'a1': float,
        ' a2': float,
        ' Average Total Energy [K]': float,
        ' Standard Deviation [K]': float,
        ' Energy per particle [erg]': float
    })
    if " -nan(ind)" in data.values:
        print(f"Warning: NaN values found in {filename}")
    data.rename(columns={
        'a1': 'a1',
        ' a2': 'a2',
        ' Average Total Energy [K]': 'Et [K]',
        ' Standard Deviation [K]': 'std [K]',
        ' Energy per particle [erg]': 'e [erg]'
    }, inplace=True)

    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(inplace=True)
    data["std [erg]"] = data["std [K]"] * 1.380649e-16/200 # Kb* erg/J /Np

    return data

In [ ]:
data = read_data('ave_energy_p200_1.csv' )
data = pd.concat([data, read_data('ave_energy_p200_0p01_0p1.csv' )], ignore_index=True)
data = pd.concat([data, read_data('ave_energy200.csv' )], ignore_index=True)

data = data[ data["e [erg]"] < 4e-15]

In [ ]:
data.sort_values(by=['e [erg]'])

In [ ]:
data["a1"].diff().unique()

In [ ]:
McMillan65_data = pd.read_csv("McMillan65_data.csv")
McMillan65_data.sort_values(by='a1 [ang]', inplace=True)

fig, ax = plt.subplots()

markers = {10: "^", 8: "o", 6: "s", 5: "D", 4: "v"}
groups = {t: [] for t in markers.keys()}

for (a1, a2, E) in McMillan65_data.values:
    for t in markers.keys():
        if np.isclose(a2, t):
            groups[t].append((a1, E))
            break

for t in markers.keys():
    xs, ys = zip(*groups[t])
    ax.plot(xs, ys, marker=markers[t], label=f"a2={t}")


ax.set_xlim(2, 3.5)
ax.set_ylim(-0.9, 0.5)
ax.grid(True)
ax.legend()

In [ ]:
data.hist(column='e [erg]', bins=10)
plt.xlabel('Energy per particle [erg]')
plt.ylabel('Count')

In [ ]:
compatible = (-0.86e-15 < data['e [erg]']) & (data['e [erg]']< -0.68e-15)
data[compatible].plot.scatter(x='a1', y='a2', c='e [erg]', cmap='viridis')

In [ ]:
compatible = (data['e [erg]']< -0.77e-15)
data[compatible].plot.scatter(x='a1', y='a2', c='e [erg]', cmap='viridis', grid=True)

In [ ]:
x = data["a1"]
y = data["a2"]
z = data["e [erg]"]*1e15
err = data["std [erg]"]*1e15/np.sqrt(data.shape[0])

In [ ]:
idx = np.argmin(z)
print(f"Minimum energy per particle: {z[idx]:.3f}+-{err[idx]:.3f} x10^{-15} erg at a1={x[idx]:.3f}, a2={y[idx]:.3f}")

In [ ]:
# At the top of your notebook
%matplotlib qt

# Create 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot triangulated surface
ax.plot_trisurf(x, y, z, cmap='viridis', alpha=0.8, edgecolor='none')


ax.plot(McMillan65_data.values[:,0], McMillan65_data.values[:,1], McMillan65_data.values[:,2], 'ro')


ax.set_xlabel(r'$a_1$')
ax.set_ylabel(r'$a_2$')
ax.set_zlabel(r'Energy per particle [$10^{15}$erg]')



plt.show()